# Aspirador de Pó

**Problema:** Implemente um simulador de ambiente de medição de desempenho para o mundo do aspirador de pó. Sua implementação deve ser modular, de forma que os sensores, os atuadores e as características do ambiente (tamanho, localização da sujeira, ... ) possam ser alterados com facilidade

a) Medida de Desempenho A medida de desempenho oferece o prêmio de um ponto para cada sala limpa em cada período de tempo ao longo de uma duração de mil períodos de tempo.

b) Conhecimento a priori A geografia do ambiente não é conhecida a priori (Figura 2.2), a distribuição da sujeira e a posição inicial do agente não são previamente conhecidas. O ambiente para o agente é parcialmente observável. A aspiração limpa o quadrado atual. As ações esquerda e direita movem o agente para a esquerda e para a direita, exceto quando isso tenta levar o agente para fora do ambiente; nesse caso, o agente permanece onde está. 

c) Ações do agente Esquerda, Direita, Limpar e NoOp (fazer nada).

d) Percepções O agente percebe corretamente sua posição e se esta posição contém sujeita.

fig 2.7

Ambiente um mundo do aspirador de pó com apenas duas salas. Estas que por sua vez, podem estar limpas ou não conforme o tempo passa. A sujeira pode surgir de modo espontâneo por fins de simulação.

# Configurações básicas do programa

In [1]:
# Logs >>>
import logging
import os
from colorlog import ColoredFormatter

"""
Dictionary that store all possible floor state values
"""
FLOOR_STATE = {"clean" : 0, "dirty" : 1}
AGENT_STATE = {"right": 1, "left": -1, "clean": 2, "NoOp": 0}


def init_log(log_level):
    # Initializing the logger
    logFormat = "%(message)s"
    formatter = ColoredFormatter("%(log_color)s" + logFormat)
    
    # Terminal
    stream = logging.StreamHandler()
    stream.setFormatter(formatter)

    logging.basicConfig(
        level=log_level,
        format=logFormat,
        handlers=[
            stream
        ]
    )



In [2]:
class Agent:
    pass

# Agente Burro

In [3]:
import random
import typing

class DumbAgent(Agent):

    def __init__(self, location: int):
        # Default init
        self.performance = 0

        # IDK ABOUT THAT
        self.direction = 1
        
        self.location = location
        
    def act(self, env):
        if env.rooms[self.location] == FLOOR_STATE["dirty"]:
            env.rooms[self.location] = FLOOR_STATE["clean"]
            
            return AGENT_STATE["clean"]
        else:
            self.direction = random.choice([AGENT_STATE["right"], AGENT_STATE["left"]])

            if self.direction == AGENT_STATE["left"] and self.location == 0:
                self.direction = AGENT_STATE["NoOp"]
            
            elif self.direction == AGENT_STATE["right"] and self.location == len(env.rooms) - 1:
                self.direction = AGENT_STATE["NoOp"]

            self.location += self.direction
        return self.direction


# Classe do Agente Inteligente

Classe responsável por armazenar e saber se comportar dado o ambiente

In [4]:
import random
import typing
import logging

class SmartAgent(Agent):

    def __init__(self, location: int):
        # Default init
        self.performance = 0

        # IDK ABOUT THAT
        self.direction = 1
        
        self.location = location
        
        
    def act(self, env):
        if env.rooms[self.location] == FLOOR_STATE["dirty"]:
            env.rooms[self.location] = FLOOR_STATE["clean"]
            
            logging.debug("Limpando a sala " + str(self.location))
            
            return AGENT_STATE["clean"]
        else:
            if self.location == 0:
                self.direction = AGENT_STATE["right"]
            elif self.location == len(env.rooms) - 1:
                self.direction = AGENT_STATE["left"]
            else:
                right = 0
                left = 0
                # Encontrar a sujeira mais próxima
                for i in range(self.location - 1, -1, -1):
                    left += 1
                    
                    if env.rooms[i] == FLOOR_STATE["dirty"]:
                        break
                for i in range(self.location + 1, len(env.rooms)):
                    right += 1
                    if env.rooms[i] == FLOOR_STATE["dirty"]:
                        break
                
                self.direction = 1 if right < left else -1
            
            logging.debug("Movendo de " + str(self.location) + " para " + str(self.location + self.direction))
            self.location += self.direction
            
            
        return self.direction


# Ambiente
O ambiente será representado a partir de um vetor 1D (Pois possui apenas ações de direita, esquerda, limpar, nada). Esse vetor pode aumentar e diminuir, não importando o tamanho inicial

In [5]:
import random
import typing
from typing import Callable
from typing import Union

class Environment:

    def __init__(self, rooms: Union[int, list], dirty_probability: int, agent: Agent):
        # 0 - clean / 1 - dirty
        if type(rooms) is int:
            self.rooms = [random.choice(list(FLOOR_STATE.values())) for i in range(0, rooms)]
        else:
            self.rooms = rooms.copy()
        
        self.size = len(self.rooms)
        

        self.agent = agent

        # Artificial Env Controller
        self.dirty_probability = dirty_probability
    
    
    def floor_is_clean(self):
        
        for floor in self.rooms:
            if floor == FLOOR_STATE["dirty"]:
                return False
        return True

    
    def update(self):
        
        choice = self.agent.act(self)

        if choice == AGENT_STATE["clean"]:
            self.agent.performance += 1
        
        elif choice is AGENT_STATE["right"] or choice is AGENT_STATE["left"]:
            self.agent.performance -= 1
        # Raise an exception ????????
        
        return


    def artificial_env_changes(self):
        
        # Rand to decide if we should dirty someplace
        if random.randint(0, 100) < self.dirty_probability: 
            self.rooms[random.randrange(0, self.size)] = 1
    
    
    def __str__(self):
        
        message = ""
        for i in range(len(self.rooms)):
            show = str(i) + ":C" if self.rooms[i] == FLOOR_STATE["clean"] else str(i) + ":D"
            message += " | " + show
        
        message += " | -> agent position: " + str(self.agent.location)
        return message


In [6]:
def evaluate(env_config, agent_position, dirty_probability, times):
    
    env = Environment(env_config, dirty_probability, agent_position)
    
    for i in range(0, times):
        # Play
        logging.debug(env)
        
        env.update()
        # Generate dirty
        env.artificial_env_changes()
    
    return env.agent.performance

def main():

    performancesSmart = []
    performancesDumb = []

    env_possible = [[0, 0], [0, 1], [1, 0], [1, 1]]
    agent_possible = [0, 1]

    init_log(logging.INFO)

    logging.info("Dumb Agent: ")
    for env_config in env_possible:
        for agent_position in agent_possible:
            performancesDumb.append(evaluate(env_config, DumbAgent(agent_position), 50, 1000))
            logging.info(f"Passos: 1000, Pontuação: {performancesDumb[-1]}, Sala: {env_config}, Inicial: {agent_position}")
    logging.info("Média da pontuação: " + str(sum(performancesDumb) / len(performancesDumb) ))


    logging.info("Smart Agent: ")
    for env_config in env_possible:
        for agent_position in agent_possible:
            performancesSmart.append(evaluate(env_config, SmartAgent(agent_position), 50, 1000))
            logging.info(f"Passos: 1000, Pontuação: {performancesSmart[-1]}, Sala: {env_config}, Inicial: {agent_position}")
    logging.info("Média da pontuação: " + str(sum(performancesSmart) / len(performancesSmart) ))

main()

Dumb Agent: 
Passos: 1000, Pontuação: 72, Sala: [0, 0], Inicial: 0
Passos: 1000, Pontuação: 82, Sala: [0, 0], Inicial: 1
Passos: 1000, Pontuação: 91, Sala: [0, 1], Inicial: 0
Passos: 1000, Pontuação: 67, Sala: [0, 1], Inicial: 1
Passos: 1000, Pontuação: 48, Sala: [1, 0], Inicial: 0
Passos: 1000, Pontuação: 75, Sala: [1, 0], Inicial: 1
Passos: 1000, Pontuação: 94, Sala: [1, 1], Inicial: 0
Passos: 1000, Pontuação: 43, Sala: [1, 1], Inicial: 1
Média da pontuação: 71.5
Smart Agent: 
Passos: 1000, Pontuação: 174, Sala: [0, 0], Inicial: 0
Passos: 1000, Pontuação: 188, Sala: [0, 0], Inicial: 1
Passos: 1000, Pontuação: 196, Sala: [0, 1], Inicial: 0
Passos: 1000, Pontuação: 179, Sala: [0, 1], Inicial: 1
Passos: 1000, Pontuação: 188, Sala: [1, 0], Inicial: 0
Passos: 1000, Pontuação: 171, Sala: [1, 0], Inicial: 1
Passos: 1000, Pontuação: 212, Sala: [1, 1], Inicial: 0
Passos: 1000, Pontuação: 183, Sala: [1, 1], Inicial: 1
Média da pontuação: 186.375
